In [2]:
import pandas as pd
import numpy as np
import rasterio
import time
import random

In [3]:
df_insert = pd.DataFrame({'NDVI_2014': [np.NaN]*7,
                    'NDVI_2015': [np.NaN]*7})
def dataframe_extraction(r1,r2,row,col):
    """This function returns a pandas dataframe that holds the NDVI values for 1-365 doys of 2015 and 2016 years.
    inputs: r1, r2: the NDVI Rasterstacks for 2015 and 2016 years. Each raster has 365 layers one corresponding to a doy of that year.
            row, col: the row and column of the raster pixel we would like to get the dataframe for """
    id=np.array(range(1,1+r1.shape[0]))
    data_frame = (pd.DataFrame({'NDVI_2014':r1[0:r1.shape[0]+1:1,row,col],
                          'NDVI_2015':r2[0:r1.shape[0]+1:1,row,col]},index=id))
    df=pd.DataFrame()
    for i in range(len(data_frame)):
        df=pd.concat([df,data_frame.iloc[[i]],df_insert])
    df.index=range(1,369)
    return df[:365]

In [4]:
"""with rasterio.open('/Users/koutilya/Downloads/MOD09A1.h10v04.brdf_corrected/MOD09A1.A2015113.h10v04.brdf_product.02.01.tif') as src:
    print("CRS: ", src.crs)
    print("Band Count: ",src.count)   #Band count
    print("Indexes: ", src.indexes)
    print("Raster width:", src.width)
    print("Raster height:", src.height)
    print("DTypes of the Raster: ", src.dtypes)
    print("Extent of the Raster: ", src.bounds)
    print("Transformation from points to XYZ: ", src.transform)
    print("sample coordinate of the left uppermost point: ", src.transform * (0, 0))#row=0 column=0
    a=src.read()
    print("Total Count: ", (a.size))
    print(a.shape)
    print(a[0,1200,1200])
src=rasterio.open('MOD09A1.A2015113.h10v04.brdf_product.02.01.tif')
a=src.read()
print(a.shape)
k=rasterio.open('MOD09A1.A2016113.h10v04.brdf_product.02.01.tif')
b=k.read()
print(dataframe_extraction(a,b,1200,1200))"""

'with rasterio.open(\'/Users/koutilya/Downloads/MOD09A1.h10v04.brdf_corrected/MOD09A1.A2015113.h10v04.brdf_product.02.01.tif\') as src:\n    print("CRS: ", src.crs)\n    print("Band Count: ",src.count)   #Band count\n    print("Indexes: ", src.indexes)\n    print("Raster width:", src.width)\n    print("Raster height:", src.height)\n    print("DTypes of the Raster: ", src.dtypes)\n    print("Extent of the Raster: ", src.bounds)\n    print("Transformation from points to XYZ: ", src.transform)\n    print("sample coordinate of the left uppermost point: ", src.transform * (0, 0))#row=0 column=0\n    a=src.read()\n    print("Total Count: ", (a.size))\n    print(a.shape)\n    print(a[0,1200,1200])\nsrc=rasterio.open(\'MOD09A1.A2015113.h10v04.brdf_product.02.01.tif\')\na=src.read()\nprint(a.shape)\nk=rasterio.open(\'MOD09A1.A2016113.h10v04.brdf_product.02.01.tif\')\nb=k.read()\nprint(dataframe_extraction(a,b,1200,1200))'

In [5]:
import os
import pdb

import numpy as np
import pandas as pd
from scipy.optimize import leastsq


def model_fourier(params, agdd, n_harm):
    """
    Fourier model
    :param params:
    :param agdd:
    :param n_harm:
    :return:
    """
    integration_time = len(agdd)
    t = np.arange(1, integration_time + 1)
    result = t*.0 + params[0]
    w = 1

    for i in range(1, n_harm * 4, 4):
        result = result + params[i] * np.cos(2.0 * np.pi * w * t / integration_time + params[i+1]) \
                 + params[i+2]*np.sin(2.0 * np.pi * w * t / integration_time + params[i+3])
        w += 1

    return result


def mismatch_function(params, func_phenology, ndvi, agdd):
    """
    The NDVI/Phenology model mismatch function
    :param params:
    :param func_phenology:
    :param ndvi:
    :param agdd:
    :param years:
    :return:
    """
    # output stores the predictions
    output = []

    oot = ndvi - func_phenology(params, agdd, n_harm=8)

    [output.append(x) for x in oot]

    return np.array(output).squeeze()


def do_fourier(ndvi, gdd, n_harm=8, init_params=None):
    """
    :param ndvi:
    :param gdd:
    :param n_harm:
    :param init_params:
    :return:
    """
    n_params = 1 + n_harm * 4

    if init_params is None:
        init_params = [.25, ] * n_params
        (xsol, mesg) = leastsq(mismatch_function, init_params, args=(model_fourier, ndvi, gdd), maxfev=1000000)
        model_fitted = model_fourier(xsol, gdd, n_harm)

    return model_fitted


def get_PTD(df,gl,gu,sl,su):
    """
    Get phenological transition dates (greenup, senescence)
    :param df:
    :return:
    """
    # Input dataframe has an index comprised of day of year and remaining columns signify NDVI
    # Linearly interpolate dataframe columns to fill in missing values
    df = df.apply(pd.Series.interpolate)
    
    # Now compute mean of all columns and get the smoothened NDVI
    arr_smooth = do_fourier(df.mean(axis=1), [8.0] * len(df))
    
    plt.plot(list(range(len(arr_smooth))),arr_smooth)
    plt.xlabel("DOY")
    plt.ylabel("NDVI")
    plt.title("NDVI smoothened time series plot")
    plt.show()
       
    # For all other crops and regions, take differential
    # To get doy_green, find the last occurrence of the max differential
    diff_green = np.diff(arr_smooth[:365 + 1])
    
    
    plt.plot(list(range(len(diff_green))),diff_green)
    plt.xlabel("DOY")
    plt.ylabel("NDVI_diff")
    plt.title("NDVI differential time series plot")
    plt.show()
    
    
    """greenup_diffvalues=diff_green[10:60]
    sen_diffvalues=diff_green[125:]
    doy_green = np.where(diff_green == diff_green[gl:gu+1].max())[0][-1]
    #doy_senesc = sen_diffvalues.argmin()
    #doy_green = np.where(diff_green == diff_green.max())[0][-1]
    doy_senesc = np.diff(arr_smooth[:365 + 1]).argmin()
    """
    green_indices=np.where(diff_green == diff_green[gl:gu+1].max())[0]
    for i in green_indices:
        if i<=gu and i>=gl:
            doy_green = i
            break
    sen_indices=np.where(diff_green == diff_green[sl:su+1].min())[0]
    for i in sen_indices:
        if i<=su and i>=sl:
            doy_senesc = i
            break
    return doy_green, doy_senesc

In [6]:
#from pathos.multiprocessing import ProcessingPool as Pool
from multiprocessing import Pool
import matplotlib.pyplot as plt
%matplotlib inline
def initialize_rasters(path1,path2):
    raster1=rasterio.open(path1)
    tot_cols=raster1.width
    tot_rows=raster1.height
    a=raster1.read()
    print("Tot rows: ",tot_rows," Tot cols: ",tot_cols)
    raster2=rasterio.open(path2)
    b=raster2.read()
    if(b.shape[0]<a.shape[0]):
        t1=b[0:6] #determine what doy you are missing!! in this case its DOY49 thus insert as 7th layer
        t2=b[6:b.shape[0]]
        p=b[0]*0
        p=p.reshape(1,tot_rows,tot_cols)

        tp=np.append(t1,p,axis=0)
        b=np.append(tp,t2,axis=0)
        #print(b.shape)
    return (a,b,tot_rows,tot_cols)
def ritvik_fn(df,gl,gu,sl,su):
    #return (random.randint(1,50),random.randint(120,365))
    return get_PTD(df,gl,gu,sl,su)

def myfunction(index,rasters):
    a=rasters[0]
    b=rasters[1]
    tot_cols=a.shape[2]
    tot_rows=a.shape[1]
    #print("Tot rows: ",tot_rows," Tot cols: ",tot_cols)
    row=int(index/tot_cols)
    col=index-(tot_cols*row)
    print("row: ",row," col: ",col)
    df=dataframe_extraction(a,b,row,col)
    print(df)
    #df.loc[57]['NDVI_2015']=None
    df[df<0]=None
    ndvi_2015=df['NDVI_2015'].tolist()
    clean=[x for x in ndvi_2015 if str(x) != 'nan']
    clean = [max(0, min(x, 10000)) for x in clean]
    maxi=int(max(clean))
    if (maxi>2000 and len(clean)>0):
        plt.plot(list(range(len(clean))),clean)
        plt.xlabel("DOY")
        plt.ylabel("NDVI")
        plt.title("NDVI time series plot")
        plt.show()
        #print(df.loc[50:80]['NDVI_2015'])
        ##return ritvik_fn(pd.Series.to_frame(df['NDVI_2015']))
        return ritvik_fn(pd.Series.to_frame(df.loc[0:365]['NDVI_2015']),10,60,135,196)
    else:
        return (0,0)

The doy interval we need to consider for the dataframe passed into ritvik_fn changes from crop to crop and ste to state. For eg, for winter wheat in Kansas, greep up happens in 1:60 doys and sennescence in 125:200 doys.

In [11]:
#a,b,tot_rows,tot_cols=initialize_rasters("A2015_177_ndvi_480m.tif","A2015_185_ndvi_480m.tif")
#src=rasterio.open('ndvi_stack_2015_area_wheat.tif')
#a,b,tot_rows,tot_cols=initialize_rasters('ndvi_stack_2014_area_wheat.tif','ndvi_stack_2015_area_wheat.tif')
src=rasterio.open('MOD09Q1.A2012.NE.BRDF_ndvistack_corn.tif')
a,b,tot_rows,tot_cols=initialize_rasters('MOD09Q1.A2012.NE.BRDF_ndvistack_corn.tif','MOD09Q1.A2012.NE.BRDF_ndvistack_soy.tif')

y=[(a,b)]*tot_rows*tot_cols
ind=range(tot_rows*tot_cols)
l=list()
l=list(list(zip(ind,y))[:])
greenup=a[1]*0
sen=a[1]*0
plant=a[1]*0
har=a[1]*0
plant=plant.astype('int32')
har=har.astype('int32')

Tot rows:  757  Tot cols:  1521


In [12]:
index_test=988650+500+6
gr,se=myfunction(l[index_test][0],l[index_test][1])
print(gr,se)

row:  650  col:  506
     NDVI_2014  NDVI_2015
1       2190.0        0.0
2          NaN        NaN
3          NaN        NaN
4          NaN        NaN
5          NaN        NaN
6          NaN        NaN
7          NaN        NaN
8          NaN        NaN
9       2101.0        0.0
10         NaN        NaN
11         NaN        NaN
12         NaN        NaN
13         NaN        NaN
14         NaN        NaN
15         NaN        NaN
16         NaN        NaN
17      2196.0        0.0
18         NaN        NaN
19         NaN        NaN
20         NaN        NaN
21         NaN        NaN
22         NaN        NaN
23         NaN        NaN
24         NaN        NaN
25      2058.0        0.0
26         NaN        NaN
27         NaN        NaN
28         NaN        NaN
29         NaN        NaN
30         NaN        NaN
..         ...        ...
336        NaN        NaN
337     2163.0        0.0
338        NaN        NaN
339        NaN        NaN
340        NaN        NaN
341        NaN   

In [70]:
start=time.time()
with Pool(processes=3) as pool:
    ind_start=988650+500
    ind_end=ind_start+100
    pairs=pool.starmap(myfunction,l[ind_start:ind_end])
    pool.close()
    pool.join()
end=time.time()
print(end-start)
#print(pairs)

137.79208493232727


In [71]:
print(pairs)

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (58, 72), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (14, 73), (14, 73), (14, 73), (0, 0), (14, 72), (0, 0), (14, 73), (14, 73), (13, 72), (13, 72), (13, 72), (13, 28), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (14, 73), (14, 73), (0, 0), (13, 72), (13, 72), (0, 0), (13, 72), (0, 0), (0, 0), (0, 0), (14, 72), (14, 72), (0, 0), (0, 0), (0, 0), (14, 72), (0, 0), (0, 0), (14, 72), (14, 72), (14, 72), (0, 0), (0, 0), (14, 73), (0, 0), (40, 73), (0, 0), (0, 0), (59, 73), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (18, 73), (0, 0), (0, 0), (0, 0), (16, 73), (0, 0), (0, 0), (0, 0), (0, 0), (15, 73), (53, 72), (0, 0), (15, 73), (0, 0), (0, 0), (14, 73), (14, 73), (14, 73), (0, 0), (0, 0), (0, 0), (21, 73), (26, 71), (0, 0), (0, 0), (0, 0), (0, 0)]


In [293]:
start=time.time()
#print(pairs)
for index in list(range(ind_start,ind_end)):
    row=int(index/tot_cols)
    col=index-(tot_cols*row)
    greenup[row][col]=pairs[index-ind_start][0]
    plant[row][col]=pairs[index-ind_start][0]-15
    #print("Row: ",row," Col: ",col," ",greenup[row][col])
    sen[row][col]=pairs[index-ind_start][1]
    har[row][col]=pairs[index-ind_start][1]+45

[(23, 153), (28, 151), (25, 149), (34, 149), (0, 0), (0, 0)]


In [294]:
np.clip(plant, 1, 365, out=plant)
np.clip(har, 1, 365, out=har)
plant=plant.astype('uint32')
har=har.astype('uint32')
print(time.time()-start)

3.3580799102783203


In [295]:
"""
print(greenup[105][400:500])
print(plant[105][400:500])
print(sen[105][400:500])
print(har[105][400:500])
"""

'\nprint(greenup[105][400:500])\nprint(plant[105][400:500])\nprint(sen[105][400:500])\nprint(har[105][400:500])\n'

In [297]:
profile=src.profile
profile.update(count=1)
print(profile)

with rasterio.open('greenup.tif', 'w', **profile) as dst:
    dst.write(greenup.astype(rasterio.float64), 1)
with rasterio.open('sen.tif', 'w', **profile) as dst:
    dst.write(sen.astype(rasterio.float64), 1)
with rasterio.open('planting.tif', 'w', **profile) as dst:
    dst.write(plant.astype(rasterio.float64), 1)
with rasterio.open('harvesting.tif', 'w', **profile) as dst:
    dst.write(har.astype(rasterio.float64), 1)

{'interleave': 'pixel', 'height': 75, 'nodata': -1.7e+308, 'dtype': 'float64', 'transform': Affine(480.0, 0.0, -184680.723585027,
       0.0, -480.0, 1589872.385601348), 'count': 1, 'tiled': False, 'crs': CRS({'wktext': True, 'lat_2': 45.5, 'ellps': 'GRS80', 'no_defs': True, 'lat_1': 29.5, 'lat_0': 23, 'units': 'm', 'lon_0': -96, 'x_0': 0, 'y_0': 0, 'towgs84': '0,0,0,0,0,0,0', 'proj': 'aea'}), 'compress': 'lzw', 'width': 104, 'driver': 'GTiff'}
